# Pipeline

In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pickle as pkl
from page_rank import *
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from utils import *
from xgboost import XGBClassifier

## Building dataset

Getting train data

In [2]:
train = pd.read_csv('data/train.txt', sep = ' ', names = ['node1', 'node2', 'is_linked'])

Getting node information

In [3]:
node_info = pd.read_csv('data/node_information.csv', header = None)

In [4]:
#node_info = node_info.rename(columns = {0 : 'id'})
col_names = {0 : 'id', '0' : 'id'}
for i, col in enumerate(node_info.columns):
    if i > 0:
        col_names[i] = str(i)

node_info = node_info.rename(columns = col_names)

Computing the graph

In [5]:
graph = nx.Graph()

for _, row in train.iterrows():
    if row['is_linked'] == 1:
        graph.add_edge(row['node1'], row['node2'], capacity = 1)

Adding node features

In [6]:
df = train.merge(node_info, how = 'inner', left_on = ['node1'], right_on = ['id'])
df = df.drop(['id'], axis = 1)
df = df.merge(node_info, how = 'inner', left_on = ['node2'], right_on = ['id'], suffixes = ('_1', '_2'))
df = df.drop(['id'], axis = 1)

Adding max flow (computation is heavy so we load the precomputed flows)

In [7]:
max_flow_data = pd.read_csv('data/cache/train_max_flow.csv')

In [8]:
df = df.merge(max_flow_data[['node1', 'node2', 'max_flow']], how = 'left', on = ['node1', 'node2'])

Adding page rank

In [9]:
page_rank_res = page_rank(graph)

In [10]:
df['page_rank_sum'] = df.apply(lambda x: page_rank_res[x['node1']] + page_rank_res[x['node2']], axis = 1)
df['page_rank_diff'] = df.apply(lambda x: np.abs(page_rank_res[x['node1']] - page_rank_res[x['node2']]), axis = 1)

Adding common neighbors overlap

In [11]:
df['common_neighbors'] = df.apply(lambda x: len(list(nx.common_neighbors(graph, x['node1'], x['node2']))), 
                                   axis = 1)

Adding degrees

In [12]:
df['degree_sum'] = df.apply(lambda x: nx.degree(graph, x['node1']) + nx.degree(graph, x['node2']), axis = 1)
df['degree_diff'] = df.apply(lambda x: np.abs(nx.degree(graph, x['node1']) - nx.degree(graph, x['node2'])), axis = 1)

Adding Jaccard coefficient

In [13]:
df['jaccard'] = df.apply(lambda x: len(set(graph.neighbors(x['node1'])) & set(graph.neighbors(x['node2']))) / len(set(graph.neighbors(x['node1'])) | set(graph.neighbors(x['node2']))), axis = 1)

Adding Adammic / Adar coefficient

In [14]:
df['adamic_adar'] = df.apply(lambda x: adamic_adar(graph, x['node1'], x['node2']), axis = 1)

Computing KatzB measure using matrix formulation

In [15]:
# betas = [0.11, 0.53, 9.87]
# betas_names = ['11', '53', '87']

In [16]:
# for beta, beta_name in zip(betas, betas_names):
#     M, katzB_index, _ = katzB_matrix(graph, beta)
#     col_name = f"katzB{beta_name}"
#     df[col_name] = df.apply(lambda x: M[katzB_index[x['node1']], katzB_index[x['node2']]], axis = 1) 

In [17]:
#df['katzB'] = df.apply(lambda x: M[katzB_index[x['node1']], katzB_index[x['node2']]], axis = 1) 

Adding shortest path length

In [18]:
df['shortest_path_length'] = df.apply(lambda x: shortest_path(graph, x['node1'], x['node2']), axis = 1)

In [ ]:
df[['node1', 'node2', 'shortest_path_length']].to_csv('data/cache/train_shortest_path', sep = ',', index = True)

In [218]:
df.sample(3)

,node1,node2,is_linked,1_1,2_1,3_1,4_1,5_1,6_1,7_1,...,932_2,max_flow,page_rank_sum,page_rank_diff,common_neighbors,degree_sum,degree_diff,jaccard,adamic_adar,shortest_path_length
2016,2360,5560,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5,0.003016,0.001888,0,32,20,0.0,0.0,1
5376,6134,7423,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,0.000475,0.000093,0,5,1,0.0,0.0,1
1140,3940,7098,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,0.000281,0.000094,0,3,1,0.0,0.0,1


Scaling features with standard scaler

In [94]:
# scaler = StandardScaler()
# scaler.fit(df)

StandardScaler()

In [95]:
# cols = df.columns
# df = scaler.transform(df)
# df = pd.DataFrame(df, columns = cols)

## Train / Test split

In [219]:
train_set, test_set = train_test_split(df, test_size = 0.2)

In [220]:
X_train, y_train = train_set.drop(['is_linked'], axis = 1), train_set['is_linked']
X_test, y_test = test_set.drop(['is_linked'], axis = 1), test_set['is_linked']

## Training model

Logistic Regression

In [ ]:
reg_log = LogisticRegression()
reg_log.fit(X_train, y_train)

Gradient Boosting

In [178]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

XGBoost

In [221]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

Decision Tree

In [213]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

Save model

In [211]:
pkl.dump(, open('models//model.pkl', 'wb'))

In [156]:
model = pkl.load(open('models//model.pkl', 'rb'))

## Evaluating model

In [223]:
y_pred = xgb.predict(X_test)

In [224]:
confusion_matrix(y_test, y_pred)

array([[1041,    0],
       [   0, 1059]], dtype=int64)

In [225]:
print(f"Accuracy : {accuracy_score(y_test, y_pred)}")
print(f"Precision : {precision_score(y_test, y_pred)}")
print(f"Recall : {recall_score(y_test, y_pred)}")

Accuracy : 1.0
Precision : 1.0
Recall : 1.0


## Predicting on test set

In [199]:
test = pd.read_csv('data/test.txt', sep = ' ', names = ['node1', 'node2'])

In [200]:
df_test = test.merge(node_info, how = 'left', left_on = ['node1'], right_on = ['id'])
df_test = df_test.drop(['id'], axis = 1)
df_test = df_test.merge(node_info, how = 'left', left_on = ['node2'], right_on = ['id'], suffixes = ('_1', '_2'))
df_test = df_test.drop(['id'], axis = 1)

In [201]:
max_flow_data = pd.read_csv('data/cache/test_max_flow.csv')
df_test = df_test.merge(max_flow_data[['node1', 'node2', 'max_flow']], how = 'left', on = ['node1', 'node2'])

In [202]:
df_test['page_rank_sum'] = df_test.apply(lambda x: page_rank_res[x['node1']] + page_rank_res[x['node2']], axis = 1)
df_test['page_rank_diff'] = df_test.apply(lambda x: np.abs(page_rank_res[x['node1']] - page_rank_res[x['node2']]), axis = 1)

In [203]:
df_test['common_neighbors'] = df_test.apply(lambda x: len(list(nx.common_neighbors(graph, x['node1'], x['node2']))), 
                                   axis = 1)

In [204]:
df_test['degree_sum'] = df_test.apply(lambda x: nx.degree(graph, x['node1']) + nx.degree(graph, x['node2']), axis = 1)
df_test['degree_diff'] = df_test.apply(lambda x: np.abs(nx.degree(graph, x['node1']) - nx.degree(graph, x['node2'])), axis = 1)

In [205]:
df_test['jaccard'] = df_test.apply(lambda x: len(set(graph.neighbors(x['node1'])) & set(graph.neighbors(x['node2']))) / len(set(graph.neighbors(x['node1'])) | set(graph.neighbors(x['node2']))), axis = 1)

In [206]:
df_test['adamic_adar'] = df_test.apply(lambda x: adamic_adar(graph, x['node1'], x['node2']), axis = 1)

In [209]:
test['Predicted'] = xgb.predict(df_test)
test.loc[test.node1 == test.node2, 'Predicted'] = 1

## Write submission

In [210]:
test.to_csv('data/submissions/10.csv', sep = ',', columns = ['Predicted'],
            index = True, index_label = 'ID')